### Apriori算法
#### 关联规则的三个基本概念
* Support
* Confidence
* Lift

In [26]:
from numpy import *
 
def loadDataSet():
    return [{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

dataSet=loadDataSet()
dataSet

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [27]:
def createC1(dataSet):
    '''根据输入的数据集，返回商品集合C1，对C进行去重排序，数据类型为frozenset'''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

C1=createC1(dataSet)
print(C1)

[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]


In [43]:
def scanD(D, Ck, minSupport):
    '''计算数据集Ck在数据集D中的支持度，并返回大于最小支持度(minSupport)的数据,和所有子项的conf字典
    '''
    ssCnt = {}      #商品规则集：出现次数
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can,0)+1   #若没有此商品规则集，将其置为1
    numItems = len(D)       #原数据集计数
    Lk= []              #候选集项Cn生成的频繁项集Lk
    supportData = {}    #候选集项Cn的支持度字典
    #计算候选项集的支持度, supportData key:候选项， value:支持度
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            Lk.append(key)          #这里返回大于支持度的频繁集
        supportData[key] = support   #这里会保存所有后端集的支持度
    return Lk,supportData

L1,support=scanD(dataSet,C1,minSupport=0.5)
print(L1)
print()
print(support)

[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]

{frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75}


In [44]:
def aprioriGen(Lk, k):  #根据Lk,生成下一层的可能的频繁集
    '''
    LK：频繁集列表LK
    项集元素个数k
    '''
    Ck = []
    lenLk = len(Lk)
    for i in range(lenLk):
        L1 = list(Lk[i])[:k - 2]
        L1.sort()
        for j in range(i + 1, lenLk):    #前k-2个项相同时，将两个集合合并，
            L2 = list(Lk[j])[:k - 2]   #可以查看生成图理解为什么是前k-2项
            L2.sort()
            if L1 == L2:
                Ck.append(Lk[i] | Lk[j])
    return Ck

print(aprioriGen(L1,2))

[frozenset({1, 3}), frozenset({1, 2}), frozenset({1, 5}), frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5})]


In [30]:
def apriori(dataSet, minSupport = 0.5):
    '''根据打她Set，和最小支持度生成可能的频繁集，和所有频繁集对应的速配Pro数据
    '''
    C1 = createC1(dataSet)
    L1, supportData = scanD(dataSet, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):  #初始时k-2为L的最后一个元素，这里K-2随K进行递增获取列表的最后的一个元素
        Lk_1 = L[k-2]           #获取L列表中的最后一组频繁集
        Ck = aprioriGen(Lk_1, k)       #生成新的可能的频繁集
        Lk, supK = scanD(dataSet, Ck, minSupport)    #返回Ck中的频繁集和所有item-support字典
        supportData.update(supK)     #更新此次的item-support到字典
        L.append(Lk)               #添加频繁集
        k += 1

    return L, supportData

l1,s1=apriori(dataSet,0.7)
print(l1)
print()
print(s1)

[[frozenset({3}), frozenset({2}), frozenset({5})], [frozenset({2, 5})], []]

{frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75, frozenset({2, 3}): 0.5, frozenset({3, 5}): 0.5, frozenset({2, 5}): 0.75}


In [31]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    '''# 计算是否满足最小可信度,如果满足更新数据'''
    prunedH = []
    #用每个conseq作为后件
    for conseq in H:
        # 计算置信度
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        lift = supportData[freqSet] / (supportData[freqSet - conseq]* supportData[conseq])
        if conf >= minConf:
            #print(freqSet - conseq, '-->', conseq, 'conf:', conf,'lift',lift)
            # 元组中的四个元素：前件、后件、conf,lift
            brl.append((freqSet - conseq, conseq, conf,lift)) #更新规则项
            prunedH.append(conseq)    
    return prunedH   #返回后件列表



def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    '''freqSet:频繁集
    H：可能成为右件的单个元素
    supportData:频繁项-支持度
    br1:当前生成的关联规则
    minConf：最小置信度
    '''
    m = len(H[0]) 
    if (len(freqSet) > (m + 1)):     #查看是否有可能生成大于1的右件，此方法已经假设关联规则的size>=3
        Hmp1 = aprioriGen(H, m + 1)   #生成可能的右件项
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  #计算置信度并更新br1，返回此次新增的右件
        if (len(Hmp1) > 0):     #如果此次后件列表存在，继续生成下一层关联规则
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            

def generateRules(L, supportData, minConf=0.7):
    '''生成关联规则
    L: 频繁项集列表
    supportData: 包含频繁项集支持数据的字典
    minConf:最小置信度
    '''
    bigRuleList = []  #初始化关联规则列表
    for i in range(1, len(L)):  #从频繁集中size=2的频繁项开始遍历
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]   #频繁集中的频繁项列表
            if (i > 1):  #如果频繁集的size大于2
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:  #如果频繁集的size=2
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)   
    return bigRuleList

#### 测试算法

In [32]:
data=loadDataSet()
L,supportData=apriori(data,0.5)
generateRules(L,supportData,0.7)

[(frozenset({1}), frozenset({3}), 1.0, 1.3333333333333333),
 (frozenset({5}), frozenset({2}), 1.0, 1.3333333333333333),
 (frozenset({2}), frozenset({5}), 1.0, 1.3333333333333333)]

In [33]:
generateRules(L,supportData,0.5)

[(frozenset({3}), frozenset({1}), 0.6666666666666666, 1.3333333333333333),
 (frozenset({1}), frozenset({3}), 1.0, 1.3333333333333333),
 (frozenset({3}), frozenset({2}), 0.6666666666666666, 0.8888888888888888),
 (frozenset({2}), frozenset({3}), 0.6666666666666666, 0.8888888888888888),
 (frozenset({5}), frozenset({3}), 0.6666666666666666, 0.8888888888888888),
 (frozenset({3}), frozenset({5}), 0.6666666666666666, 0.8888888888888888),
 (frozenset({5}), frozenset({2}), 1.0, 1.3333333333333333),
 (frozenset({2}), frozenset({5}), 1.0, 1.3333333333333333),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666, 1.3333333333333333),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666, 0.8888888888888888),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666, 1.3333333333333333)]

#### 挖掘Groceries数据集中的关联规则

In [41]:
GroceriesData=[]
with open('Groceries.csv','r') as f:
    for i in range(9835):
        line=str(f.readline())
        start_index=line.index('{')
        end_index=line.index('}')
        text=line[start_index+1:end_index]
        item=text.split(',')
        GroceriesData.append(item)

In [42]:
L,supportData=apriori(GroceriesData,0.01)
generateRules(L,supportData,0.4)

[(frozenset({'yogurt'}),
  frozenset({'whole milk'}),
  0.40160349854227406,
  1.5717351405345263),
 (frozenset({'butter'}),
  frozenset({'whole milk'}),
  0.4972477064220184,
  1.9460530014566455),
 (frozenset({'tropical fruit'}),
  frozenset({'whole milk'}),
  0.40310077519379844,
  1.5775949558420244),
 (frozenset({'curd'}),
  frozenset({'whole milk'}),
  0.4904580152671756,
  1.9194805332879712),
 (frozenset({'root vegetables'}),
  frozenset({'other vegetables'}),
  0.43470149253731344,
  2.2466049285887952),
 (frozenset({'hamburger meat'}),
  frozenset({'other vegetables'}),
  0.41590214067278286,
  2.149446954028807),
 (frozenset({'sugar'}),
  frozenset({'whole milk'}),
  0.4444444444444445,
  1.7393995666976168),
 (frozenset({'root vegetables'}),
  frozenset({'whole milk'}),
  0.44869402985074625,
  1.75603095247994),
 (frozenset({'whipped/sour cream'}),
  frozenset({'whole milk'}),
  0.449645390070922,
  1.7597542424781207),
 (frozenset({'whipped/sour cream'}),
  frozenset({'ot

### FP-Tree算法

#### 1.创建FP-tree数据结构

In [45]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name = nameValue   #当前节点的名称，使用商品item进行命名
        self.count =numOccur       
        self.nodeLink =None   #当前节点的headerTable链表
        self.parent =parentNode  #节点的父节点
        self.children = {}  #当前节点的子节点
        
    def inc(self,numOccur):
        self.count += numOccur  #发生次数
        
    def disp(self,ind =1):  #结构化打印树
        print(' '*ind,self.name,' ',self.count)
        
        for child in self.children.values():
            child.disp(ind+1)
            
rootNode=treeNode('pyramid',9,None)
rootNode.children['eye']=treeNode('eye',13,None)
rootNode.disp()

  pyramid   9
   eye   13


In [46]:
rootNode.children['phoenix']=treeNode('phoenix',3,None)
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


In [47]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):      #将原数据的格式更新，便于更新树
    retDict = {}
    for trans in dataSet:
        fset = frozenset(trans)
        retDict.setdefault(fset, 0)
        retDict[fset] += 1
    return retDict

In [51]:
def createTree(dataSet,minSup=1):
    '''根据dataSet遍历两个数据库，创建FP-tree,
    第一次，生成headerTable,记录频繁项-频繁项出现的次数
    第二次，生成FP-tree，并将headerTable中的item指针指向树中对应的层级节点'''
    headerTable={}   #初始化headerTable
    for trans in dataSet:  #遍历交易
        for item in trans:  #遍历交易中的item
            headerTable[item]=headerTable.get(item,0)+ 1   #遍历计算每个item的count
            
    lessThanMinsup = list(filter(lambda k:headerTable[k] < minSup, headerTable.keys()))
    for k in lessThanMinsup: 
        del(headerTable[k])   #删除不满足最小支持度的item
        
    freItemSet=set(headerTable.keys())         #获取频繁item的集合
    
    if len(freItemSet)==0:
        return None,None   #如果所有的item都不满足最小支持度，返回None,None
    
    for k in headerTable.keys():               #格式转化,headerTable的每一项第一个元素为count，
        headerTable[k]=[headerTable[k],None]    #第二个元素为指针指向node

        
    retTree=treeNode('φ', 1, None)     #构建树的根节点，开始第二次遍历数据集，构建FP-tree
    for tranSet,count in dataSet.items():   
        localD={}               #初始化这条交易的item,和此item出现的次数
        for item in tranSet:
            if item in freItemSet:
                localD[item]=headerTable[item][0]     #获取对应item出现的次数，下一步排序时需要
                
        if len(localD)>0:     #如果此条tran中的频繁项大于0，进行排序
            orderItems=[v[0] for v in 
            sorted(localD.items(),key=lambda p:(p[1],p[0]),reverse=True)] #将此条trans的item排序
            updateTree(
                orderItems,retTree,headerTable,count)  #更新树,orderItems为排序后的trans,count为相同的trans出现次数
       
    return retTree,headerTable
    
def updateTree(items, inTree, headerTable, count):

    if items[0] in inTree.children:  # 检查items的第一个元素是否位于根节点之下
        inTree.children[items[0]].inc(count)  # 更新树中item的count
    else:  # 如果不位于根节点之下
        inTree.children[items[0]] = treeNode(items[0], count, inTree)  #将其添加在根节点下
        if headerTable[items[0]][1] == None:  # 如果headerTable之前不存在指针，添加指针
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:  #如果headerTable存在指针，使用更新header方法连接新节点
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])

    if len(items) > 1:  # 如果items的size大于1，迭代调用
        updateTree(items[1:], inTree.children[items[0]], headerTable, count)


def updateHeader(nodeToTest, targetNode): 
    '''如果headertable的指针已经存在，不断迭代，直至
    找到None节点
    '''
    while (nodeToTest.nodeLink != None):  
        nodeToTest = nodeToTest.nodeLink  #获取下下一级目标，直到目标节点为None
    nodeToTest.nodeLink = targetNode
    
    
simpDat = loadSimpDat()
dictDat = createInitSet(simpDat)
myFPTree,myheader = createTree(dictDat, 3)
myFPTree.disp()            

  φ   1
   z   5
    r   1
    x   3
     y   3
      t   3
       s   2
       r   1
   x   1
    s   1
     r   1


#### 2.从FP-tree中挖掘频繁项

In [161]:
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

def findPrefixPath(basePat, headTable):
    condPats = {}
    treeNode = headTable[basePat][1]
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

def mineTree(inTree, headerTable, minSup=1, preFix=set([]), freqItemList=[]):
    # order by minSup asc, value asc
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: (p[1][0],p[0]))]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        # 通过条件模式基找到的频繁项集
        condPattBases = findPrefixPath(basePat, headerTable)
        myCondTree, myHead = createTree(condPattBases, minSup)
        if myHead != None:
            print('condPattBases: ', basePat, condPattBases)
            myCondTree.disp()
            print('*' * 30)

            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [162]:
simpDat = loadSimpDat()
dictDat = createInitSet(simpDat)
myFPTree,myheader = createTree(dictDat, 3)
myFPTree.disp()
condPats = findPrefixPath('z', myheader)
print('z', condPats)
condPats = findPrefixPath('x', myheader)
print('x', condPats)
condPats = findPrefixPath('y', myheader)
print('y', condPats)
condPats = findPrefixPath('t', myheader)
print('t', condPats)
condPats = findPrefixPath('s', myheader)
print('s', condPats)
condPats = findPrefixPath('r', myheader)
print('r', condPats)

mineTree(myFPTree, myheader, 2)

  φ   1
   z   5
    r   1
    x   3
     y   3
      t   3
       s   2
       r   1
   x   1
    s   1
     r   1
z {}
x {frozenset({'z'}): 3}
y {frozenset({'z', 'x'}): 3}
t {frozenset({'y', 'z', 'x'}): 3}
s {frozenset({'y', 't', 'z', 'x'}): 2, frozenset({'x'}): 1}
r {frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'y', 't', 'z', 'x'}): 1}
condPattBases:  r {frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'y', 't', 'z', 'x'}): 1}
  φ   1
   z   2
    x   1
   x   1
******************************
condPattBases:  s {frozenset({'y', 't', 'z', 'x'}): 2, frozenset({'x'}): 1}
  φ   1
   x   3
******************************
